In [1]:
# import os and define graphic card
import os
os.environ["OMP_NUM_THREADS"] = "1"

# import common libraries
import gc
import random
import argparse
import pandas as pd
import numpy as np
from functools import partial

# import pytorch related libraries
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.optimizer import Optimizer
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, _LRScheduler
from tensorboardX import SummaryWriter
from pytorch_pretrained_bert.optimization import BertAdam
from transformers import get_linear_schedule_with_warmup

# import apex for mix precision training
from apex import amp
from apex.parallel import DistributedDataParallel as DDP
from apex.optimizers import FusedAdam

# import dataset class
from dataset.dataset import *

# import utils
from utils.ranger import *
from utils.lrs_scheduler import * 
from utils.loss_function import *
from utils.metric import *
from utils.file import *

# import model
from model.model_bert import *

I0130 21:39:40.255348 140695671617280 file_utils.py:38] PyTorch version 1.2.0 available.
I0130 21:39:41.047496 140695671617280 file_utils.py:54] TensorFlow version 2.0.0 available.


matplotlib.get_backend :  TkAgg


In [6]:
model = QuestNet(model_type="flaubert-large-cased", n_classes=30, hidden_layers=[-1, -3, -5, -7, -9])

I0130 21:52:33.749969 140695671617280 filelock.py:274] Lock 140691829489168 acquired on /home/jionie/.cache/torch/transformers/97248896d50e5f71a0bca4f6e45cecc29b3802056a252f08353d40c5d5597fca.431d47beb46d4c737166d5f81333822462ba18e64d76b60314e08f43840e86dc.lock
I0130 21:52:33.753069 140695671617280 file_utils.py:413] https://s3.amazonaws.com/models.huggingface.co/bert/flaubert/flaubert_large_cased/config.json not found in cache or force_download set to True, downloading to /home/jionie/.cache/torch/transformers/tmpvjjb4_1m


I0130 21:52:34.004426 140695671617280 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/flaubert/flaubert_large_cased/config.json in cache at /home/jionie/.cache/torch/transformers/97248896d50e5f71a0bca4f6e45cecc29b3802056a252f08353d40c5d5597fca.431d47beb46d4c737166d5f81333822462ba18e64d76b60314e08f43840e86dc
I0130 21:52:34.005129 140695671617280 file_utils.py:426] creating metadata file for /home/jionie/.cache/torch/transformers/97248896d50e5f71a0bca4f6e45cecc29b3802056a252f08353d40c5d5597fca.431d47beb46d4c737166d5f81333822462ba18e64d76b60314e08f43840e86dc
I0130 21:52:34.006445 140695671617280 filelock.py:318] Lock 140691829489168 released on /home/jionie/.cache/torch/transformers/97248896d50e5f71a0bca4f6e45cecc29b3802056a252f08353d40c5d5597fca.431d47beb46d4c737166d5f81333822462ba18e64d76b60314e08f43840e86dc.lock
I0130 21:52:34.007272 140695671617280 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/

I0130 21:53:24.606858 140695671617280 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/flaubert/flaubert_large_cased/pytorch_model.bin in cache at /home/jionie/.cache/torch/transformers/434a5601ffb94eb010f8d1ea979f8082a9849b44907798693b07116341d88c48.711513e7dc80b480c11c365177d07ecafe2606c83ba675767ed4a9168af1e1bf
I0130 21:53:24.607414 140695671617280 file_utils.py:426] creating metadata file for /home/jionie/.cache/torch/transformers/434a5601ffb94eb010f8d1ea979f8082a9849b44907798693b07116341d88c48.711513e7dc80b480c11c365177d07ecafe2606c83ba675767ed4a9168af1e1bf
I0130 21:53:24.608362 140695671617280 filelock.py:318] Lock 140691829489000 released on /home/jionie/.cache/torch/transformers/434a5601ffb94eb010f8d1ea979f8082a9849b44907798693b07116341d88c48.711513e7dc80b480c11c365177d07ecafe2606c83ba675767ed4a9168af1e1bf.lock
I0130 21:53:24.608781 140695671617280 modeling_utils.py:458] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/flaub

In [7]:
model.cuda()

QuestNet(
  (flaubert_model): FlaubertModel(
    (position_embeddings): Embedding(512, 1024)
    (embeddings): Embedding(68729, 1024, padding_idx=2)
    (layer_norm_emb): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
    (attentions): ModuleList(
      (0): MultiHeadAttention(
        (q_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (k_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (v_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (out_lin): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (1): MultiHeadAttention(
        (q_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (k_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (v_lin): Linear(in_features=1024, out_features=1024, bias=True)
        (out_lin): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (2): MultiHeadAttention(
        (q_lin): Linear(in_features=1024, out_features=